In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import climlab

/Users/br546577/miniconda3/envs/climlab-courseware/lib/python3.10/site-packages/climlab/convection/akmaev_adjustment.py:142: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  Akmaev_adjustment = jit(signature_or_function=Akmaev_adjustment)


Test some of the simpler parts of the module

In [2]:
# Do these two methods for calculating saturation vapor pressure agree?

import _simplified_betts_miller
from climlab.utils.thermo import clausius_clapeyron

print(clausius_clapeyron(280.)) 
print(_simplified_betts_miller.escomp(280.)/100.)

9.911891305211338
9.911895751953125


In [15]:
#  Calculate CAPE and CIN for a simple column profile
from climlab.utils import constants as const

Cp_air = const.cp 
rdgas = const.Rd 
rvgas = const.Rv
HLv = const.Lhvap 
kappa = const.kappa 
es0 = 1.0
avgbl = True
Grav = const.g

num_lev = 30
water_depth = 10.

full_state = climlab.column_state(water_depth=water_depth, num_lev=num_lev)
# set initial conditions -- 24C at the surface, -60C at 200 hPa, isothermal stratosphere
strat_idx = 6
full_state['Tatm'][:strat_idx] = -60. + const.tempCtoK
full_state['Tatm'][strat_idx:] = np.linspace(-60., 42., num_lev-strat_idx) + const.tempCtoK
full_state['Ts'][:] = 44. + const.tempCtoK
full_state['Tatm']
#  Initialize a nearly dry column (small background stratospheric humidity)
full_state['q'] = 0.*full_state.Tatm + 5.E-6
temperature_state = {'Tatm':full_state.Tatm,'Ts':full_state.Ts}

pfull = full_state.Tatm.domain.axes['lev'].points[np.newaxis,np.newaxis,:] * 100
phalf = full_state.Tatm.domain.axes['lev'].bounds[np.newaxis,np.newaxis,:] * 100
tin = full_state.Tatm[np.newaxis,np.newaxis,:]
qin = full_state.q[np.newaxis,np.newaxis,:]

#  Call the calccalc routine
# !    Output:
# !    cape        Convective available potential energy
# !    cin         Convective inhibition (if there's no LFC, then this is set 
# !                to zero)
# !    tp          Parcel temperature (set to the environmental temperature 
# !                where no adjustment)
# !    rp          Parcel specific humidity (set to the environmental humidity 
# !                where no adjustment, and set to the saturation humidity at 
# !                the parcel temperature below the LCL)
# !    klzb        Level of zero buoyancy
cape,cin,tp,rp,klzb = _simplified_betts_miller.capecalcnew(num_lev,pfull,phalf,
                        Cp_air,rdgas,rvgas,HLv,kappa,es0,tin,qin,avgbl)

print(cape, cin, klzb)

0.0 0.0 0


In [16]:
# Check to see if the parcel temperature is different from initial temperature
tp - tin

Field([[[-6.10351560e-06, -6.10351560e-06, -6.10351560e-06,
         -6.10351560e-06, -6.10351560e-06, -6.10351560e-06,
         -6.10351560e-06, -4.77666438e-06, -3.44981316e-06,
         -2.12296192e-06, -7.96110726e-07,  5.30740522e-07,
          1.85759174e-06,  3.18444296e-06,  4.51129418e-06,
          5.83814540e-06,  7.16499665e-06,  8.49184784e-06,
          9.81869908e-06,  1.11455503e-05,  1.24724015e-05,
          1.37992528e-05,  1.51261040e-05, -1.40646230e-05,
         -1.27377717e-05, -1.14109205e-05, -1.00840693e-05,
         -8.75721804e-06, -7.43036685e-06, -6.10351560e-06]]])

In [17]:
# Now add moisture
saturation = climlab.utils.thermo.qsat(full_state['Tatm'], full_state['Tatm'].domain.axes['lev'].points)
full_state['q'][10:] = 0.8*saturation[10:]
cape,cin,tp,rp,klzb = _simplified_betts_miller.capecalcnew(num_lev,pfull,phalf,
                        Cp_air,rdgas,rvgas,HLv,kappa,es0,tin,qin,avgbl)
print(cape, cin, klzb)

23434.041015625 -0.6649989485740662 3


In [6]:
tp - tin

Field([[[-6.10351560e-06, -1.32115082e+01,  1.64967285e+01,
          3.30463654e+01,  4.32079407e+01,  5.03613220e+01,
          5.58666931e+01,  5.59133558e+01,  5.52649807e+01,
          5.41142252e+01,  5.25832512e+01,  5.07543035e+01,
          4.86853351e+01,  4.64187654e+01,  4.39865768e+01,
          4.14135190e+01,  3.87190623e+01,  3.59188928e+01,
          3.30257667e+01,  3.00502736e+01,  2.70012637e+01,
          2.38862138e+01,  2.07114714e+01,  1.74825299e+01,
          1.42041193e+01,  1.08803902e+01,  7.51497405e+00,
          4.11110574e+00,  6.71592911e-01, -6.78289795e-01]]])

So at 80% relative humidity throughout the troposphere, there is positive CAPE and negative CIN in this sounding.

### Call the convection module

In [18]:
dt = const.seconds_per_hour * 3
tau_bm=7200.
rhbm=0.8
do_simp=True
do_shallower=False
do_changeqref=False
do_envsat=False
do_taucape=False
capetaubm=900.
tau_min=2400.
ix = 1; jx = 1; kx = num_lev

rain, tdel, qdel, q_ref, bmflag, klzbs, cape, cin, t_ref, invtau_bm_t, invtau_bm_q, capeflag = \
    _simplified_betts_miller.betts_miller(dt, tin, qin, pfull, phalf, HLv, Cp_air, Grav,
                                          rdgas,rvgas,kappa, es0, tau_bm, rhbm, 
                                          do_simp, do_shallower, do_changeqref, 
                                          do_envsat, do_taucape, capetaubm, tau_min, 
                                          ix, jx, kx,)

In [19]:
cape

array([[28433.598]], dtype=float32)

In [20]:
rain

array([[0.]], dtype=float32)

There is positive CAPE but no rain is generated here.

## Coupling to climlab processes

In [24]:
# We have wrapped the SBM code in a climlab process:
from simplified_betts_miller import SimplifiedBettsMiller

In [25]:
from climlab.surface import SensibleHeatFlux, LatentHeatFlux
from climlab.radiation import RRTMG, DailyInsolation, AnnualMeanInsolation
from climlab import couple
from climlab.utils import constants as const

In [32]:
num_lev = 30
water_depth = 10.
short_timestep = const.seconds_per_hour * 3
long_timestep = short_timestep*3
insolation = 360. #345.
albedo = 0.18

full_state = climlab.column_state(water_depth=water_depth, num_lev=num_lev)
# set initial conditions -- 24C at the surface, -60C at 200 hPa, isothermal stratosphere
strat_idx = 6
full_state['Tatm'][:strat_idx] = -60. + const.tempCtoK
full_state['Tatm'][strat_idx:] = np.linspace(-60, 22, num_lev-strat_idx) + const.tempCtoK
full_state['Ts'][:] = 24. + const.tempCtoK
full_state['Tatm']
#  Initialize a nearly dry column (small background stratospheric humidity)
full_state['q'] = 0.*full_state.Tatm + 5.E-6
saturation = climlab.utils.thermo.qsat(full_state['Tatm'], full_state['Tatm'].domain.axes['lev'].points)
full_state['q'] = 0.8 * saturation
temperature_state = {'Tatm':full_state.Tatm,'Ts':full_state.Ts}
#  Surface model
shf = SensibleHeatFlux(name='Sensible Heat Flux',
                       state=temperature_state, Cd=3E-3,
                       timestep=short_timestep)
lhf = LatentHeatFlux(name='Latent Heat Flux',
                     state=full_state, Cd=3E-3,
                     timestep=short_timestep)
surface = couple([shf,lhf], name="Slab")
#  Convection scheme -- water vapor is a state variable
conv = SimplifiedBettsMiller(name='Convection',
                         state=full_state,
                         timestep=short_timestep,
                         )  

rad = RRTMG(name='Radiation',
                state=temperature_state,
                specific_humidity=full_state.q,
                albedo=albedo,
                insolation=insolation,
                timestep=long_timestep,
                icld=0, # no clouds
                )
atm = couple([rad, conv], name='Atmosphere')
moistmodel = couple([atm,surface], name='Moist column model')

In [33]:
print(moistmodel)

climlab Process of type <class 'climlab.process.time_dependent_process.TimeDependentProcess'>. 
State variables and domain shapes: 
  Tatm: (30,) 
  Ts: (1,) 
  q: (30,) 
The subprocess tree: 
Moist column model: <class 'climlab.process.time_dependent_process.TimeDependentProcess'>
   Atmosphere: <class 'climlab.process.time_dependent_process.TimeDependentProcess'>
      Radiation: <class 'climlab.radiation.rrtm.rrtmg.RRTMG'>
         SW: <class 'climlab.radiation.rrtm.rrtmg_sw.RRTMG_SW'>
         LW: <class 'climlab.radiation.rrtm.rrtmg_lw.RRTMG_LW'>
      Convection: <class 'simplified_betts_miller.SimplifiedBettsMiller'>
   Slab: <class 'climlab.process.time_dependent_process.TimeDependentProcess'>
      Sensible Heat Flux: <class 'climlab.surface.turbulent.SensibleHeatFlux'>
      Latent Heat Flux: <class 'climlab.surface.turbulent.LatentHeatFlux'>



In [35]:
conv.compute()

{'Ts': Field([0.]),
 'Tatm': Field([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'q': Field([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])}

In [36]:
moistmodel.step_forward()

In [37]:
moistmodel.integrate_days(30)

Integrating for 240 steps, 30.000000000000004 days, or 0.08213727767492365 years.
Total elapsed time is 0.0824795163319025 years.


In [38]:
moistmodel.q

Field([5.65088984e-04, 1.88309078e-04, 1.12978979e-04, 8.06972912e-05,
       6.27637044e-05, 5.13516764e-05, 4.34511576e-05, 5.88757869e-05,
       7.98738807e-05, 1.08160494e-04, 1.45910339e-04, 1.95848737e-04,
       2.61356823e-04, 3.46591146e-04, 4.56618074e-04, 5.97563452e-04,
       7.76777969e-04, 1.00301868e-03, 1.28664716e-03, 1.63984481e-03,
       2.07684602e-03, 2.61419002e-03, 3.27099283e-03, 4.06924085e-03,
       5.03410868e-03, 6.19430430e-03, 7.58244606e-03, 9.23547712e-03,
       1.11951250e-02, 2.43050922e-02])

In [39]:
moistmodel.integrate_years(1)

Integrating for 2921 steps, 365.2422 days, or 1 years.
Total elapsed time is 1.0821586333671191 years.


In [40]:
moistmodel.Ts

Field([327.97285042])

In [41]:
moistmodel.cape

array([inf])

In [42]:
moistmodel.q

Field([5.65088984e-04, 1.88309078e-04, 1.12978979e-04, 8.06972912e-05,
       6.27637044e-05, 5.13516764e-05, 4.34511576e-05, 5.88757869e-05,
       7.98738807e-05, 1.08160494e-04, 1.45910339e-04, 1.95848737e-04,
       2.61356823e-04, 3.46591146e-04, 4.56618074e-04, 5.97563452e-04,
       7.76777969e-04, 1.00301868e-03, 1.28664716e-03, 1.63984481e-03,
       2.07684602e-03, 2.61419002e-03, 3.27099283e-03, 4.06924085e-03,
       5.03410868e-03, 6.19430430e-03, 7.58244606e-03, 9.23547712e-03,
       1.11951250e-02, 1.03880791e-01])

This runs without blowing up now, but why isn't the atmosphere getting moister above the first level?

In [43]:
conv.diagnostics

{'precipitation': array([0.]), 'cape': array([inf]), 'cin': array([0.])}

In [44]:
moistmodel.timeave

{'Tatm': Field([209.62813265, 206.3564058 , 205.59893026, 207.7182402 ,
        208.20682647, 208.87344624, 209.38297534, 209.48164405,
        209.92997055, 210.87106146, 212.30822761, 214.20811748,
        216.53966037, 219.21502437, 222.094423  , 225.0715327 ,
        228.09318026, 231.16061979, 234.39214454, 237.94805981,
        241.9686104 , 246.52239351, 251.60410917, 257.12784985,
        262.96827544, 269.00403447, 275.25112517, 281.93724645,
        289.86093587, 311.60304927]),
 'Ts': Field([317.18504283]),
 'q': Field([5.65088984e-04, 1.88309078e-04, 1.12978979e-04, 8.06972912e-05,
        6.27637044e-05, 5.13516764e-05, 4.34511576e-05, 5.88757869e-05,
        7.98738807e-05, 1.08160494e-04, 1.45910339e-04, 1.95848737e-04,
        2.61356823e-04, 3.46591146e-04, 4.56618074e-04, 5.97563452e-04,
        7.76777969e-04, 1.00301868e-03, 1.28664716e-03, 1.63984481e-03,
        2.07684602e-03, 2.61419002e-03, 3.27099283e-03, 4.06924085e-03,
        5.03410868e-03, 6.19430430e-03,